In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from database.market import Market
from database.adatabase import ADatabase
from datetime import datetime, timedelta
from tqdm import tqdm
from extractor.tiingo_extractor import TiingoExtractor
from extractor.forex_extractor import FOREXExtractor
from time import sleep

In [2]:
market = Market()
sp500 = ADatabase("sp500")
market.connect()
sp5 = market.retrieve("sp500")
market.disconnect()

In [3]:
market.connect()
start = market.retrieve_max_date("stocks")
market.disconnect()
end = datetime.now().strftime("%Y-%m-%d")
print(start,end)

2023-06-10 2023-06-13


In [4]:
market.connect()
data = FOREXExtractor.extract(start,end)
values = pd.DataFrame(data["rates"].values.tolist())
dates = data["rates"].keys().tolist()
values["dates"]= dates
market.store("forex",values)
market.disconnect()

In [5]:
values

,AED,AFN,ALL,AMD,ANG,AOA,ARS,AUD,AWG,AZN,...,XCD,XDR,XOF,XPF,YER,ZAR,ZMK,ZMW,ZWL,dates
0,3.673042,85.625377,98.184014,385.304909,1.801496,665.503981,244.303562,1.483680,1.8000,1.703970,...,2.70255,0.751475,609.314012,111.450364,250.350363,18.713685,9001.203589,20.117041,321.999592,2023-06-10
1,3.672970,85.625377,98.184014,385.304909,1.801496,665.499323,244.576184,1.482655,1.8000,1.698309,...,2.70255,0.751475,609.314012,111.449930,250.350295,18.716850,9001.202143,20.117041,321.999592,2023-06-11
2,3.672896,87.499420,98.450073,386.399178,1.802634,676.499789,246.734197,1.480867,1.8025,1.748038,...,2.70255,0.750837,607.504668,111.301736,250.350063,18.558430,9001.212630,19.729164,321.999592,2023-06-12
3,3.672896,85.760743,98.741682,386.978058,1.804284,676.496871,246.724902,1.478450,1.8025,1.695814,...,2.70255,0.751545,609.602803,111.303576,250.349863,18.524098,9001.194452,19.747499,321.999592,2023-06-13


In [6]:
market.connect()
for ticker in tqdm(list(sp5["Symbol"].unique())):
    try:
        if "." in ticker:
            ticker = ticker.replace(".","-")
        try:
            data = TiingoExtractor.extract(ticker,start,end)
            data["ticker"] = ticker
            market.store("stocks",data)
        except:
            print(ticker,"tiingo")
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

  1%|█▋                                                                                                                                                                         | 5/490 [00:03<05:09,  1.57it/s]

market stocks documents must be a non-empty list


 73%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                             | 358/490 [03:33<01:18,  1.68it/s]

market stocks documents must be a non-empty list


 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                 | 441/490 [04:22<00:29,  1.66it/s]

market stocks documents must be a non-empty list


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [04:52<00:00,  1.68it/s]


In [7]:
analysis = []
market.connect()
periods = ["year","quarter","month","week"]
for ticker in tqdm(sp5["Symbol"].unique()):
    report = {}
    try:
        test = market.retrieve_ticker_prices("stocks",ticker)
        test["date"] = pd.to_datetime(test["date"])
        test["year"] = [x.year for x in test["date"]]
        test = test[test["year"]==test["year"].max()].sort_values("date")      
        test["quarter"] = [x.quarter for x in test["date"]]
        test["week"] = [x.week for x in test["date"]]
        test["month"] = [x.month for x in test["date"]]
        ep = test.tail(1)["adjClose"].item()
        for period in periods:
            current = test.tail(1)[period].item()
            window = test[(test[period]==current)].sort_values("date")
            sp = window.iloc[0]["adjClose"].item()
            gain = (ep - sp) / sp
            report[f"{period}_gain"] = gain
        report["day_gain"] = (ep - test.tail(2).iloc[0]["adjClose"].item())/test.tail(2).iloc[0]["adjClose"].item()
        report["ticker"] = ticker
        analysis.append(report)
    except Exception as e:
        print(ticker,str(e))
market.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [00:32<00:00, 15.28it/s]


In [8]:
period = "day"
a = pd.DataFrame(analysis)
a = a.merge(sp5[["Symbol","Security","GICS Sector","GICS Sub-Industry"]].rename(columns={"Symbol":"ticker"}),on="ticker")
a.sort_values(f"{period}_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain,ticker,Security,GICS Sector,GICS Sub-Industry
86,0.846926,0.476429,0.242194,0.0,0.124523,CCL,Carnival,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
88,-0.058746,-0.320089,0.156216,0.0,0.102293,CTLT,Catalent,Health Care,Pharmaceuticals
333,0.552277,0.379026,0.186976,0.0,0.072219,NCLH,Norwegian Cruise Line Holdings,Consumer Discretionary,"Hotels, Resorts & Cruise Lines"
75,0.556586,0.331362,0.082803,0.0,0.063061,AVGO,Broadcom Inc.,Information Technology,Semiconductors
344,0.401812,0.244961,0.098500,0.0,0.059900,ORCL,Oracle Corporation,Information Technology,Application Software
241,0.252926,0.005473,0.062319,0.0,0.055201,INTC,Intel,Information Technology,Semiconductors
10,-0.553226,-0.448170,-0.016757,0.0,0.052060,AAP,Advance Auto Parts,Consumer Discretionary,Automotive Retail
312,0.537475,0.067448,0.053937,0.0,0.048519,MPWR,Monolithic Power Systems,Information Technology,Semiconductors
47,0.406004,0.013473,0.019918,0.0,0.046025,ANET,Arista Networks,Information Technology,Communications Equipment
8,0.408732,0.248763,0.112197,0.0,0.045441,ADBE,Adobe Inc.,Information Technology,Application Software


In [9]:
a.groupby("GICS Sector").mean().sort_values("quarter_gain",ascending=False).head(20)

,year_gain,quarter_gain,month_gain,week_gain,day_gain
GICS Sector,,,,,
Information Technology,0.191774,0.043988,0.029218,0.000000,0.021474
Industrials,0.085868,0.026637,0.056278,0.000000,0.007570
Real Estate,-0.000481,0.016920,0.051549,0.000000,0.000622
Consumer Discretionary,0.116995,0.014114,0.064785,0.000000,0.014702
Communication Services,0.111635,0.010195,0.043094,0.001840,0.007218
Financials,-0.062524,0.001738,0.040349,0.000000,-0.007926
Health Care,-0.001924,-0.010166,0.020182,-0.001451,0.011107
Consumer Staples,0.006455,-0.014274,0.007325,0.000000,0.000190
Utilities,-0.039521,-0.015258,0.032538,0.000000,-0.000660
